Straight forward method to extract relevant details using string pattern matching
Its easy to implement, but need proper string pattern mapping for this to work thus its less robust solution for the problem.

In [29]:
import re
from pdfminer.high_level import extract_pages, extract_text
import pdfplumber
import pickle as pickle
import numpy as np
import os

def extract_details_from_pdf(pdf_path):
    # Extract text from the PDF
    text = extract_text(pdf_path)
    invoice_details = {}

    # Use regular expressions to find the relevant information
    account_pattern = r"Account #:\s*(\d+)"
    ifsc_pattern = r"IFSC Code:\s*([A-Z0-9]+)"
    bank_pattern = r"Bank:\s*([A-Za-z\s]+)"
    branch_pattern = r"Branch:\s*([A-Za-z\s-]+)"

    # Search for the patterns in the extracted text
    account_match = re.search(account_pattern, text)
    ifsc_match = re.search(ifsc_pattern, text)
    bank_match = re.search(bank_pattern, text)
    branch_match = re.search(branch_pattern, text)

    # Store the matched groups in the dictionary
    if account_match:
        invoice_details['Account'] = account_match.group(1)
    if ifsc_match:
        invoice_details['IFSC'] = ifsc_match.group(1)
    if bank_match:
        invoice_details['Bank'] = bank_match.group(1).split("\n")[0]
    if branch_match:
        invoice_details['Branch'] = branch_match.group(1).split("\n")[0]


    gstin_pattern = r"GSTIN\s*([\dA-Z]+)"
    address_pattern = r"C/o\s*([\w\s,.-]+)"
    mobile_pattern = r"Mobile\s*\+?(\d{2}\s*\d{10})"
    email_pattern = r"Email\s*([\w\.-]+@[\w\.-]+)"
    invoice_number_pattern = r"Invoice #:\s*(\w+-\d+)"
    invoice_date_pattern = r"Invoice Date:\s*([\d\s\w]+)"
    due_date_pattern = r"Due Date:\s*([\d\s\w]+)"
    customer_pattern = r"Customer Details:\s*([\w\s]+)"
    place_of_supply_pattern = r"Place of Supply:\s*([\dA-Z\s-]+)"

    # Search for the patterns in the extracted text
    gstin_match = re.search(gstin_pattern, text)
    address_match = re.search(address_pattern, text)
    mobile_match = re.search(mobile_pattern, text)
    email_match = re.search(email_pattern, text)
    invoice_number_match = re.search(invoice_number_pattern, text)
    invoice_date_match = re.search(invoice_date_pattern, text)
    due_date_match = re.search(due_date_pattern, text)
    customer_match = re.search(customer_pattern, text)
    place_of_supply_match = re.search(place_of_supply_pattern, text)

    # Store the matched groups in the dictionary
    if gstin_match:
        invoice_details['GSTIN'] = gstin_match.group(1)
    if address_match:
        invoice_details['Address'] = ",".join(address_match.group(1).strip().split("\n")[:-1])
    if mobile_match:
        invoice_details['Mobile'] = mobile_match.group(1).split()[1]
    if email_match:
        invoice_details['Email'] = email_match.group(1)
    if invoice_number_match:
        invoice_details['Invoice Number'] = invoice_number_match.group(1)
    if invoice_date_match:
        invoice_details['Invoice Date'] = invoice_date_match.group(1).split("\n")[0]
    if due_date_match:
        invoice_details['Due Date'] = due_date_match.group(1).split("\n")[0]
    if customer_match:
        invoice_details['Customer'] = customer_match.group(1).strip().split("\n")[0]
    if place_of_supply_match:
        invoice_details['Place of Supply'] = place_of_supply_match.group(1).strip()

    #item_details stored in tables
    with pdfplumber.open(pdf_path) as pdf:
        for page_num in range(len(pdf.pages)):
            page = pdf.pages[page_num]
            
            # Extract tables from the page
            tables = page.extract_tables()
            data = tables[0]
            for item in data:
                # Split the string into the key and value parts
                key_value = item[0].rsplit(" ", 1)  # Split from the right into two parts
                key = key_value[0].strip()           # Key is everything before the last space
                value = key_value[1].strip()         # Value is the last part after the space
                invoice_details[key] = value             # Add the key-value pair to the dictionary
    return invoice_details

good part about above model is its accuracy as we are hardcoding patterns and taking details from the pdf we know for sure the accuracy is 100 percent for all the pdfs in  our training data. we can leverage this information to create unit testcases and use them to validate our remaining models. To do that lets create a pickle file of above dictionary which can be used directly in unit testing.

In [30]:
# run if invoice_details.pkl file is not present or you want to update invoice_details.pkl file 
# unit_details = {}
# # Specify the directory you want to list
# directory = 'data'  # Change this to your directory if necessary
# # List files in the specified directory
# try:
#     files = os.listdir(directory)
#     i=0
#     for filename in files:
#         detail  = extract_details_from_pdf(f'data\{filename}')
#         unit_details[detail['Invoice Number']] = detail # mapping data with invoice number for faster access based on invoice number.
#         i+=1
# except FileNotFoundError:
#     print(f"Error: The directory '{directory}' does not exist.")
# except PermissionError:
#     print(f"Error: Permission denied to access '{directory}'.")

# pickle.dump(unit_details,open('invoice_details.pkl','wb'))